In [57]:
import torch
from torch import nn

In [58]:
B, C, H, W = 1, 2, 6, 6
fold_size = 2

In [59]:
a = torch.arange(B*C*H*W*1.).reshape(B, C, H, W)
print(a.size())
a

torch.Size([1, 2, 6, 6])


tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9., 10., 11.],
          [12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23.],
          [24., 25., 26., 27., 28., 29.],
          [30., 31., 32., 33., 34., 35.]],

         [[36., 37., 38., 39., 40., 41.],
          [42., 43., 44., 45., 46., 47.],
          [48., 49., 50., 51., 52., 53.],
          [54., 55., 56., 57., 58., 59.],
          [60., 61., 62., 63., 64., 65.],
          [66., 67., 68., 69., 70., 71.]]]])

# nn.Unfold()

In [60]:
fold = nn.Unfold(kernel_size=fold_size, dilation=1, padding=0, stride=fold_size)

In [61]:
b = fold(a)
print(b.size()) # [B, C, H, W] -> [B, C*h*w, nh*nw]
b

torch.Size([1, 8, 9])


tensor([[[ 0.,  2.,  4., 12., 14., 16., 24., 26., 28.],
         [ 1.,  3.,  5., 13., 15., 17., 25., 27., 29.],
         [ 6.,  8., 10., 18., 20., 22., 30., 32., 34.],
         [ 7.,  9., 11., 19., 21., 23., 31., 33., 35.],
         [36., 38., 40., 48., 50., 52., 60., 62., 64.],
         [37., 39., 41., 49., 51., 53., 61., 63., 65.],
         [42., 44., 46., 54., 56., 58., 66., 68., 70.],
         [43., 45., 47., 55., 57., 59., 67., 69., 71.]]])

In [62]:
b.reshape(B, int(C*fold_size*fold_size), nh, nw)

tensor([[[[ 0.,  2.,  4.],
          [12., 14., 16.],
          [24., 26., 28.]],

         [[ 1.,  3.,  5.],
          [13., 15., 17.],
          [25., 27., 29.]],

         [[ 6.,  8., 10.],
          [18., 20., 22.],
          [30., 32., 34.]],

         [[ 7.,  9., 11.],
          [19., 21., 23.],
          [31., 33., 35.]],

         [[36., 38., 40.],
          [48., 50., 52.],
          [60., 62., 64.]],

         [[37., 39., 41.],
          [49., 51., 53.],
          [61., 63., 65.]],

         [[42., 44., 46.],
          [54., 56., 58.],
          [66., 68., 70.]],

         [[43., 45., 47.],
          [55., 57., 59.],
          [67., 69., 71.]]]])

## 手写unfold

In [63]:
nh = H // fold_size
nw = W // fold_size
nh, nw

(3, 3)

In [64]:
b_ = a.reshape(B, C, nh, fold_size, nh, fold_size)  # [B, C, H, W] -> [B, C, nh, h, nw, w]    H = nh * h  W = nw * w 前面代表划分数量,后面代表每份长度,不能反过来
b_ = b_.permute(0, 1, 3, 5, 2, 4)                   # [B, C, nh, h, nw, w] -> [B, C, h, w, nh, nw]
b_ = b_.reshape(B, int(C*fold_size*fold_size), -1)  # [B, C, h, w, nh, nw] -> [B, C*h*w, nh*nw]
print(b_.size())
b_

torch.Size([1, 8, 9])


tensor([[[ 0.,  2.,  4., 12., 14., 16., 24., 26., 28.],
         [ 1.,  3.,  5., 13., 15., 17., 25., 27., 29.],
         [ 6.,  8., 10., 18., 20., 22., 30., 32., 34.],
         [ 7.,  9., 11., 19., 21., 23., 31., 33., 35.],
         [36., 38., 40., 48., 50., 52., 60., 62., 64.],
         [37., 39., 41., 49., 51., 53., 61., 63., 65.],
         [42., 44., 46., 54., 56., 58., 66., 68., 70.],
         [43., 45., 47., 55., 57., 59., 67., 69., 71.]]])

In [65]:
torch.all(b == b_)

tensor(True)

# nn.Fold()

In [66]:
# output_size: new_h = new_w
# kernel_size,stride same as UnFold
fold = nn.Fold(output_size=H, kernel_size=fold_size, dilation=1, padding=0, stride=fold_size)

In [67]:
c = fold(b)
print(c.size())
c

torch.Size([1, 2, 6, 6])


tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9., 10., 11.],
          [12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23.],
          [24., 25., 26., 27., 28., 29.],
          [30., 31., 32., 33., 34., 35.]],

         [[36., 37., 38., 39., 40., 41.],
          [42., 43., 44., 45., 46., 47.],
          [48., 49., 50., 51., 52., 53.],
          [54., 55., 56., 57., 58., 59.],
          [60., 61., 62., 63., 64., 65.],
          [66., 67., 68., 69., 70., 71.]]]])

In [68]:
torch.all(a == c)

tensor(True)

## 手写fold

In [69]:
c_ = b.reshape(B, C, fold_size, fold_size, nh, nw)  # [B, C*h*w, nh*nw] -> [B, C, h, w, nh, nw]
c_ = c_.permute(0, 1, 4, 2, 5, 3)                   # [B, C, h, w, nh, nw] -> [B, C, nh, h, nw, w]
c_ = c_.reshape(1, C, H, W)                         # [B, C, nh, h, nw, w] -> [B, C, H, W]
print(c_.size())
c_

torch.Size([1, 2, 6, 6])


tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9., 10., 11.],
          [12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23.],
          [24., 25., 26., 27., 28., 29.],
          [30., 31., 32., 33., 34., 35.]],

         [[36., 37., 38., 39., 40., 41.],
          [42., 43., 44., 45., 46., 47.],
          [48., 49., 50., 51., 52., 53.],
          [54., 55., 56., 57., 58., 59.],
          [60., 61., 62., 63., 64., 65.],
          [66., 67., 68., 69., 70., 71.]]]])

In [70]:
torch.all(c == c_)

tensor(True)